In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os
from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import acquire
import env

import prepare

import graphviz
from graphviz import Graph

In [2]:
#sns.set(rc={'figure.figsize':(11.7,8.27)})

Project Objectives

Document code, process (data acquistion, preparation, exploratory data analysis and statistical testing, modeling, and model evaluation), findings, and key takeaways in a Jupyter Notebook report.

Create modules (acquire.py, prepare.py) that make your process repeateable.

Construct a model to predict customer churn using classification techniques.

In [3]:
#acquire the data

df = pd.read_sql('''SELECT *
FROM customers
JOIN contract_types USING(contract_type_id)
JOIN internet_service_types USING(internet_service_type_id)
JOIN payment_types USING(payment_type_id);''', acquire.get_connection("telco_churn"))

In [4]:
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [5]:
################################################################ PREPARE

In [6]:
#investigate the data from all angles and prepare
df.shape

(7043, 24)

In [7]:
df.describe()

,payment_type_id,internet_service_type_id,contract_type_id,senior_citizen,tenure,monthly_charges
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,2.315633,1.872923,1.690473,0.162147,32.371149,64.761692
std,1.148907,0.737796,0.833755,0.368612,24.559481,30.090047
min,1.000000,1.000000,1.000000,0.000000,0.000000,18.250000
25%,1.000000,1.000000,1.000000,0.000000,9.000000,35.500000
50%,2.000000,2.000000,1.000000,0.000000,29.000000,70.350000
75%,3.000000,2.000000,2.000000,0.000000,55.000000,89.850000
max,4.000000,3.000000,3.000000,1.000000,72.000000,118.750000


**What are some possible drivers of churn?**
Is a high monthly bill a driver of churn?
Do new customers with less tenure drive churn?
Does subscription to fiber optics drive churn?
Do customers with no partner or dependents churn more?
Do senior citizens churn more?
Does payment type effect the churn rate?

In [8]:
#drop columns with id since I used those just to JOIN the data
df.drop(columns=['payment_type_id','internet_service_type_id','contract_type_id'],inplace=True)

In [9]:
#make sure they are gone
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,0017-DINOC,Male,0,No,No,54,No,No phone service,Yes,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,Yes,Yes,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,Yes,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,No,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [10]:
#make target column binary
df.churn.replace(to_replace=['yes','no'],value=[1,0], inplace=True)

In [11]:
#drop customer_id since I have no use for it
df.drop(columns=['customer_id'], inplace=True)

In [12]:
#check to see that the target is binary and the id column is gone
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,Male,0,No,No,54,No,No phone service,Yes,No,No,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,Female,0,No,No,56,No,No phone service,Yes,Yes,Yes,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,Male,0,Yes,Yes,20,No,No phone service,Yes,No,Yes,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,Male,0,Yes,No,72,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [13]:
#drop all additional services since I am not interested in exploring
df.drop(columns=['online_security','online_backup','device_protection','tech_support','streaming_tv','streaming_movies'],inplace=True)

In [14]:
#make sure they are actually gone
df.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,Female,0,Yes,Yes,65,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,Male,0,No,No,54,No,No phone service,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,Female,0,No,No,56,No,No phone service,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,Male,0,Yes,Yes,20,No,No phone service,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,Male,0,Yes,No,72,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [15]:
#check for missing values in the total_charges column
df.total_charges.str.contains('')

0       True
1       True
2       True
3       True
4       True
        ... 
7038    True
7039    True
7040    True
7041    True
7042    True
Name: total_charges, Length: 7043, dtype: bool

In [16]:
#find any missing values
df.total_charges.value_counts()

20.2       11
           11
19.75       9
20.05       8
19.9        8
           ..
2119.5      1
551.95      1
6347.55     1
4326.8      1
156.25      1
Name: total_charges, Length: 6531, dtype: int64

In [17]:
#this shows me each row where the total_charges column have a missing vlaue
df[df['total_charges'].str.contains(' ')]

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
85,Female,0,Yes,Yes,0,No,No phone service,No,56.05,,No,Two year,DSL,Credit card (automatic)
156,Male,0,No,Yes,0,Yes,Yes,Yes,61.90,,No,Two year,DSL,Bank transfer (automatic)
236,Female,0,Yes,Yes,0,Yes,Yes,No,73.35,,No,Two year,DSL,Mailed check
255,Female,0,Yes,Yes,0,No,No phone service,Yes,52.55,,No,Two year,DSL,Bank transfer (automatic)
339,Female,0,Yes,Yes,0,Yes,No,No,80.85,,No,Two year,DSL,Mailed check
5681,Female,0,Yes,Yes,0,Yes,No,No,20.00,,No,Two year,None,Mailed check
5717,Male,0,No,Yes,0,Yes,No,No,20.25,,No,Two year,None,Mailed check
5727,Male,0,Yes,Yes,0,Yes,Yes,No,25.35,,No,Two year,None,Mailed check
5798,Male,0,Yes,Yes,0,Yes,Yes,No,25.75,,No,Two year,None,Mailed check
6007,Male,0,Yes,Yes,0,Yes,No,No,19.85,,No,Two year,None,Mailed check


In [18]:
# this will get rid of the rows with no value in the total_charges column
df.drop(df[df['total_charges'].str.contains(" ")].index, inplace = True)

In [19]:
# now I am checking that the missing values don't exist anymore
df.total_charges.value_counts()

20.2       11
19.75       9
20.05       8
19.65       8
19.9        8
           ..
2119.5      1
551.95      1
6347.55     1
4326.8      1
156.25      1
Name: total_charges, Length: 6530, dtype: int64

In [20]:
#split the dataset
prepare.prep_telco(df)

(      gender  senior_citizen partner dependents  tenure phone_service  \
 6096  Female               0     Yes         No      70           Yes   
 1603    Male               0     Yes        Yes      15           Yes   
 5350  Female               1     Yes         No      52           Yes   
 2068    Male               0      No         No      39            No   
 6366  Female               0     Yes         No      32           Yes   
 ...      ...             ...     ...        ...     ...           ...   
 296   Female               0     Yes        Yes      71           Yes   
 4200    Male               1     Yes         No      16           Yes   
 3012    Male               0     Yes         No      40           Yes   
 763   Female               0      No         No      11           Yes   
 3935  Female               0     Yes        Yes      10           Yes   
 
         multiple_lines paperless_billing  monthly_charges total_charges churn  \
 6096                No     

In [24]:
#check to see what train looks like
train.head()

NameError: name 'train' is not defined

In [25]:
#check it out after split
validate.head()

NameError: name 'validate' is not defined

In [26]:
#check out test data
test.head

NameError: name 'test' is not defined

In [ ]:
############################################################ EXPLORE

In [ ]:
plt.figure(figsize=(16, 6)) # changes the size of the figure
sns.countplot(data=train, x='tenure', hue='churn')
plt.show()

In [ ]:
# plot monthly_charges by churn
plt.figure(figsize=(20, 6))
sns.countplot(data=train, x='monthly_charges', hue='churn')
plt.show()

In [ ]:
# plot internet_service_type by churn
sns.countplot(data=train, x='internet_service_type', hue='churn')
plt.show()

In [ ]:
# plot with or without partner by churn
sns.countplot(data=train, x='partner',hue='churn')
plt.show()

In [ ]:
# plot with or without dependents by churn
sns.countplot(data=train, x='dependents', hue='churn')
plt.show()

In [ ]:
# plot senior_citizen by churn
sns.countplot(data=train, x='senior_citizen', hue='churn')
plt.show()

In [ ]:
# plot payment_type by churn
sns.countplot(data=train, x='payment_type', hue='churn')
plt.show()

In [ ]:
#let's look at some more visualizations
A = sns.regplot(x=df['churn'], y=df['monthly_charges'], fit_reg=False)

In [ ]:
sns.violinplot(x=train["churn"], y=df["tenure"])

In [ ]:
sns.violinplot(x=train["churn"], y=df["monthly_charges"])

In [ ]:
sns.regplot(x=train["tenure"], y=df["monthly_charges"])

In [ ]:
sns.regplot(x=train["tenure"], y=df["monthly_charges"])

In [ ]:
# make a correlogram
sns.pairplot(train)
sns.plt.show()

In [ ]:
# Make a random dataset:
tenure = train.tenure
churn = df.churn
y_pos = np.arange(len(churn))

# Create bars
plt.bar(y_pos, tenure)

# Create names on the x-axis
plt.xticks(y_pos, churn)

# Show graphic
plt.show()

**Takeaways from countplots**
Senior citizens churn at a much higher rate tha 